# MANTENIMIENTO PREDICTIVO EN MOTORES DE AVIÓN

# LOGISTIC REGRESION

Modelo lineal que estimar probabilidades.

Simple, rápido y fácil de interpretar.

Funciona bien si existe una relación lineal entre variables y clases

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



In [2]:
df= pd.read_csv('../data/processed/datos_norm_limpios.csv', delimiter=',')

df

,id,cycle,s2,s3,s4,s7,s8,s11,s12,s13,s14,s15,s17,s20,s21,RUL,label1,label2
0,1,1,0.183735,0.406802,0.309757,0.726248,0.242424,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662,191.0,0,0
1,1,2,0.283133,0.453019,0.352633,0.628019,0.212121,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014,190.0,0,0
2,1,3,0.343373,0.369523,0.370527,0.710145,0.272727,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375,189.0,0,0
3,1,4,0.343373,0.256159,0.331195,0.740741,0.318182,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386,188.0,0,0
4,1,5,0.349398,0.257467,0.404625,0.668277,0.242424,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502,187.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,0.686747,0.587312,0.782917,0.254428,0.439394,0.726190,0.170576,0.558824,0.194344,0.656791,0.750000,0.271318,0.109500,4.0,1,2
20627,100,197,0.701807,0.729453,0.866475,0.162641,0.500000,0.708333,0.211087,0.500000,0.188668,0.727203,0.583333,0.124031,0.366197,3.0,1,2
20628,100,198,0.665663,0.684979,0.775321,0.175523,0.515152,0.738095,0.281450,0.529412,0.212148,0.922278,0.833333,0.232558,0.053991,2.0,1,2
20629,100,199,0.608434,0.746021,0.747468,0.133655,0.530303,0.916667,0.208955,0.514706,0.203065,0.823394,0.583333,0.116279,0.234466,1.0,1,2


In [3]:
class_counts = df['label2'].value_counts()
class_counts

label2
0    17531
2     1600
1     1500
Name: count, dtype: int64

Después de ver las predicciones para label2 se intentam mejorar quitando datos en los que label2=0. Se elige quitar un porcentaje de cada ID (motor) del comienzo de los datos.

Se prueba con diferentes porcentajes y se elige lo siguiente:

In [19]:
'''
df['cumcount'] = df.groupby('id').cumcount()
df_filtered = df[df['cumcount'] >= df['cumcount'].max() * 0.2]
df_filtered = df_filtered.drop(columns='cumcount')
df_filtered
'''

"\ndf['cumcount'] = df.groupby('id').cumcount()\ndf_filtered = df[df['cumcount'] >= df['cumcount'].max() * 0.2]\ndf_filtered = df_filtered.drop(columns='cumcount')\ndf_filtered\n"

In [4]:
class_counts = df_filtered['label2'].value_counts()
class_counts

NameError: name 'df_filtered' is not defined

In [5]:
plt.figure(figsize=[30,10])
sns.heatmap (df_filtered.corr(numeric_only=True), annot=True, cmap='coolwarm', vmin=-1);

NameError: name 'df_filtered' is not defined

<Figure size 3000x1000 with 0 Axes>

In [25]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

X = df.drop(['id','cycle','RUL','label1','label2'], axis=1)
y = df['label1']  # ajusta según la etiqueta que quieras predecir
groups = df['id']

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train = groups.iloc[train_idx]



In [26]:

print(X_train.shape)
print(X_test.shape)

(16561, 13)
(4070, 13)


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

# Supongamos que ya tienes X_train, X_test, y_train, y_test (split agrupado por motor)

# Escalado manual
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000,class_weight='balanced')

# Entrenamiento
model.fit(X_train_scaled, y_train)

# Predicción
y_pred = model.predict(X_test_scaled)


c:\Users\LEIRE AL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1254: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


In [28]:

print(X_train.shape)
print(X_test.shape)

(16561, 13)
(4070, 13)


In [32]:
X_test

,s2,s3,s4,s7,s8,s11,s12,s13,s14,s15,s17,s20,s21
0,0.183735,0.406802,0.309757,0.726248,0.242424,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662
1,0.283133,0.453019,0.352633,0.628019,0.212121,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014
2,0.343373,0.369523,0.370527,0.710145,0.272727,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375
3,0.343373,0.256159,0.331195,0.740741,0.318182,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386
4,0.349398,0.257467,0.404625,0.668277,0.242424,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18510,0.533133,0.642032,0.757259,0.359098,0.545455,0.827381,0.228145,0.588235,0.101507,0.782224,0.583333,0.186047,0.317039
18511,0.813253,0.564639,0.779203,0.107890,0.606061,0.696429,0.228145,0.617647,0.107029,0.797999,0.750000,0.302326,0.153687
18512,0.906627,0.693046,0.747806,0.120773,0.590909,0.726190,0.140725,0.558824,0.104087,0.744132,0.666667,0.186047,0.151892
18513,0.750000,0.770874,0.821573,0.206119,0.560606,0.803571,0.157783,0.661765,0.098411,0.756445,0.666667,0.085271,0.160867


In [29]:
# Métricas
print(classification_report(
    y_test, y_pred,
    labels=[0,1,2],
    target_names=['nivel 0','nivel 1','nivel 2'],
    digits=3
))


              precision    recall  f1-score   support

     nivel 0      0.994     0.931     0.961      3450
     nivel 1      0.715     0.968     0.822       620
     nivel 2      0.000     0.000     0.000         0

    accuracy                          0.936      4070
   macro avg      0.570     0.633     0.595      4070
weighted avg      0.951     0.936     0.940      4070



c:\Users\LEIRE AL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\LEIRE AL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\LEIRE AL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn